In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time
import os
import scipy.stats as st
import datetime as dt
from pprint import pprint
from config import gkey
gmaps.configure(api_key=gkey)


In [12]:
#Read in the us large quakes file
us_csv="output_data/large_quakes_1980-2020.csv"
large_quakes=pd.read_csv(us_csv)
large_quakes.head()

,Lat,Lng,Depth,Mag,Sig,Place,Time,Month,Day,Year,Country
0,43.530,146.753,44.0,7.0,754,Kuril Islands,1980-02-23 05:51:03.200,2,23,1980,RU
1,-12.410,166.381,33.0,7.5,865,Santa Cruz Islands,1980-07-08 23:19:19.800,7,8,1980,SB
2,-12.525,165.916,33.0,7.9,960,Santa Cruz Islands,1980-07-17 19:42:23.200,7,17,1980,SB
3,36.195,1.354,10.0,7.3,820,northern Algeria,1980-10-10 12:25:23.500,10,10,1980,DZ
4,-21.890,169.853,33.0,7.2,798,southeast of the Loyalty Islands,1980-10-25 11:00:05.100,10,25,1980,NC


In [13]:
#Isolating largest quakes in USA
large_quakes_us = large_quakes.loc[large_quakes['Country']=="US"]
large_quakes_us


,Lat,Lng,Depth,Mag,Sig,Place,Time,Month,Day,Year,Country
5,41.117000,-124.253000,19.000,7.2,798,offshore Northern California,1980-11-08 10:27:34.000,11,8,1980,US
69,51.520000,-174.776000,33.000,8.0,985,"Andreanof Islands, Aleutian Islands, Alaska",1986-05-07 22:47:10.870,5,7,1986,US
90,58.679000,-142.786000,10.000,7.9,960,Gulf of Alaska,1987-11-30 19:23:19.590,11,30,1987,US
94,56.953000,-143.032000,10.000,7.8,936,Gulf of Alaska,1988-03-06 22:35:38.140,3,6,1988,US
106,55.321700,-156.429300,11.000,7.0,754,"164 km SE of Chignik, Alaska",1989-09-04 13:14:58.520,9,4,1989,US
134,54.567000,-161.606000,28.400,7.0,754,Alaska Peninsula,1991-05-30 13:17:41.970,5,30,1991,US
139,41.679000,-125.856000,1.303,7.0,754,"136km WSW of Brookings, Oregon",1991-08-17 22:17:09.970,8,17,1991,US
145,40.335333,-124.228667,9.856,7.2,895,"20km SSW of Rio Dell, California",1992-04-25 18:06:05.180,4,25,1992,US
150,34.200000,-116.437000,-0.097,7.3,1443,"Landers, California Earthquake",1992-06-28 11:57:34.130,6,28,1992,US
178,40.405500,-126.302833,4.972,7.0,754,off the coast of Northern California,1994-09-01 15:15:48.310,9,1,1994,US


In [14]:
# Configure gmaps
gmaps.configure(api_key=gkey)

# Store latitude and longitude in locations and convert magnitude to float
locations = large_quakes_us[["Lat", "Lng"]]
magnitude = large_quakes_us["Mag"].astype(float)


# Using the template add the earthquake marks to the gmap
info_box_template = """
<dl>
<dt>Place</dt><dd>{Place}</dd>
<dt>Mag</dt><dd>{Mag}</dd>
</dl>
"""
# Store the DataFrame Row
large_quake_info = [info_box_template.format(**row) for index, row in large_quakes_us.iterrows()]

In [16]:
fig = gmaps.figure()
# Create marker layer
markers = gmaps.marker_layer(locations, info_box_content=large_quake_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))